In [2]:
import os
import glob
import re
import pandas as pd
import numpy as np
import PyPDF2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


path = os.getcwd()
pdf_files = glob.glob(path + "/Protocolos/*.pdf")
print(len(pdf_files))

422


In [2]:
def quitar_acentos(texto):
    acentos = { 'á': 'a', 'Á': 'a', 'é': 'e', 'É': 'e', 'í': 'i', 
               'Í': 'i', 'ó': 'o', 'Ó': 'o', 'ú': 'u', 'Ú': 'u', 
               'ü': 'u', 'Ü': 'u', 'ñ': 'n', 'Ñ': 'n'
    }
    for acento, sin_acento in acentos.items():
        texto = texto.replace(acento, sin_acento)
    return texto

In [1]:
def eliminar_caracteres_especiales(texto):
        texto = re.sub(r'[•◦○◘◙◈◇◆◄►▲▼▶◀●➢]', '', texto)
        texto = re.sub(r' +', ' ', texto)
        return texto

In [4]:
def convert_pdf_to_txt(path):
    pdf_file = open(path, 'rb')
    read_pdf = PyPDF2.PdfReader(pdf_file)
    number_of_pages = len(read_pdf.pages)
    page_content = ""
    for page_number in range(number_of_pages):
        page = read_pdf.pages[page_number]
        page_content += page.extract_text()
    pdf_file.close()
    return page_content

In [5]:

text = []

print("Proceso 1")
index=1
ttborrar = []
for i in pdf_files:
    # try:
    print(f"{index}/{len(pdf_files)}", end='\r')
    text.append(convert_pdf_to_txt(i))
    index+=1
    # except:
    #     print("Error")
    #     ttborrar.append(i)
    #     index+=1
    #     # text.append("Error")


Proceso 1


Overwriting cache for 0 1743


In [6]:
profesores = pd.read_excel("profes.xlsx")
profesores["PROF"] = profesores["PROF"].apply(lambda x: re.sub(r' +', ' ', x))
profesores['PROF'] = profesores['PROF'].apply(lambda x: x.strip())
profesores.to_excel("profesores.xlsx", index=False)

In [12]:

def get_title(documento):
    if re.search(r"Trabajo\s*Terminal\s*No\.?", documento, flags = re.IGNORECASE):
        titulo = re.split(r'(?:T\s*r\s*a\s*b\s*a\s*j\s*o)\s*(?:T\s*e\s*r\s*m\s*i\s*n\s*a\s*l)?\s*(?:No)\s*\.?', documento, flags = re.IGNORECASE | re.MULTILINE)[0]
    else:
        titulo = re.split(r'(?:T\s*r\s*a\s*b\s*a\s*j\s*o)\s*(?:T\s*e\s*r\s*m\s*i\s*n\s*a\s*l)?\s*\.?', documento, flags = re.IGNORECASE | re.MULTILINE)[0]
    titulo = re.sub(r'\n|^\s*[^a-zA-ZÁÉÍÓÚÜÑáéíóúüñ]+|\s{2,}|(^\s+|\s+$)', ' ', titulo).strip()
    return titulo

def get_autores(documento):
    if re.search(r'(?:(?:estudiante|alumno|alumna|integrante)s?\s*:)', documento, flags = re.IGNORECASE):
        documento = re.split(r'(?:(?:estudiante|alumno|alumna|integrante)s?\s*:)', documento, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)[1]
    else:
        documento = re.split(r'(?:estudiantes?\s*:?|alumn[oa]?s?\s*:?|integrantes?\s*:?)', documento, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)[1]
    if re.search(r"Directo", documento, flags = re.IGNORECASE):
        autores = re.split(r'Directo', documento, maxsplit=1, flags=re.IGNORECASE)[0]
    else:
        autores = re.split(r'resumen', documento, maxsplit=1, flags=re.IGNORECASE)[0]
    if re.search(r"e-?\s*mail:?", autores, flags=re.IGNORECASE):
        autores = re.split(r"e-?\s*mail:?", autores, maxsplit=1, flags=re.IGNORECASE)[0]
    autores = re.sub(r's:', '', autores, flags=re.IGNORECASE)
    autores = re.sub(r'\n|^\s*[^a-zA-ZÁÉÍÓÚÜÑáéíóúüñ]+|\s{2,}|(^\s+|\s+$)', ' ', autores).strip()
    autores = re.sub(r'\*|\d', '', autores)
    autores = re.sub(r'^[a-záéíóúüñ]', lambda x: x.group(0).upper(), autores)
    autores = re.sub(r'\s{2,}', ' ', autores)
    autores = re.sub(r'\.', '', autores)
    autores = re.sub(r'\[|\]', '', autores)
    # si hay espacios entre dos letras minusculas, se junta
    autores = re.sub(r'(?<=[a-xzáéíóúüñ])\s(?=[a-xzáéíóúüñ])', '', autores)
    # si hay espacios entre una letra mayuscula y una minuscula, se junta
    autores = re.sub(r'(?<=[A-ZÁÉÍÓÚÜÑ])\s(?=[a-xzáéíóúüñ])', '', autores)
    autores = [autor.strip() for autor in re.split(',|;|\s+y\s+', autores)]
    # si hay elementos vacios en la lista, se eliminan
    autores = [autor for autor in autores if autor != '']
    return autores

def get_director(documento):
    try:
        if re.search(r"d\s*i\s*r\s*e\s*c\s*t\s*o\s*r\s*(?:e\s*s|a\s*s|a)?\s*:", documento, flags = re.IGNORECASE):
            documento = re.split(r"d\s*i\s*r\s*e\s*c\s*t\s*o\s*r\s*(?:e\s*s|a\s*s|a)?\s*:", documento, maxsplit=1, flags=re.IGNORECASE | re.MULTILINE)[1]
        else:
            documento = re.split(r"d\s*i\s*r\s*e\s*c\s*t\s*o\s*r\s*(?:e\s*s|a\s*s|a)?\s*:?", documento, maxsplit=1, flags=re.IGNORECASE | re.MULTILINE)[1]
        documento = re.split(r'Resumen|e\s*[-]*\s*mail|turno', documento, maxsplit=1, flags=re.IGNORECASE | re.MULTILINE)[0]
        documento = re.sub(r'\n|(\*|(\s{2,})|(^[^a-zA-ZÁÉÍÓÚÜÑáéíóúüñ]+)|(\s+$)|(\w+@[^@]+\.[a-zA-ZÁÉÍÓÚÜÑáéíóúüñ]{2,}))|\s*(?:M|Dra?)\.?\s*(?:en)?\s*(?:ing|c)?\.?', lambda m: ' ' if m.group(1) else '', documento, flags=re.IGNORECASE | re.MULTILINE)
        documento = re.sub(r'\.', '', documento)
        # si hay espacios entre dos letras minusculas, se junta
        documento = re.sub(r'(?<=[a-xzáéíóúüñ])\s(?=[a-xzáéíóúüñ])', '', documento)
        # si hay espacios entre una letra mayuscula y una minuscula, se junta
        documento = re.sub(r'(?<=[A-ZÁÉÍÓÚÜÑ])\s(?=[a-xzáéíóúüñ])', '', documento)
        documento = [i.strip() for i in re.split(',|;|\s+y\s+', documento)]
        documento = [re.sub(r'^[a-záéíóúüñ]', lambda x: x.group(0).upper(), i) for i in documento]
        directores = [director for director in documento if director != '']
        for i, director in enumerate(directores):
            if len(director.split(' ')) > 6:
                directores[i] = ' '.join(director.split(' ')[:len(director.split(' '))//2])
        vectorizer = CountVectorizer()
        for i, director in enumerate(directores):
            auxprofe = director
            auxval=0
            for profesor in profesores['PROF']:
                # quitamos acentos 
                director_aux = quitar_acentos(director)
                profesor_aux = quitar_acentos(profesor)

                vectorizer.fit([director_aux, profesor_aux])
                vector = vectorizer.transform([director_aux, profesor_aux])
                if cosine_similarity(vector)[0][1] > 0.8:
                    auxval = cosine_similarity(vector)[0][1]
                    auxprofe = profesor
                    break
                if cosine_similarity(vector)[0][1] > 0.4:
                    if cosine_similarity(vector)[0][1] > auxval:
                        auxval = cosine_similarity(vector)[0][1]
                        auxprofe = profesor
            if auxval == 0:
                # print(director_aux, " / ",auxprofe, " / ", auxval)
                directores[i] = '¡¡¡¡¡¡¡¡¡¡¡¡¡REVISAR!!!!!!!!!!!!!!!' + director
            else:
                # print(director, " / ",auxprofe, " / ", auxval)
                directores[i] = auxprofe      
        documento = directores
    except:
        documento = ['']
    finally:
        return documento

def get_ttnum(documento, ruta):
    documento = re.split(r'(?:T\s*r\s*a\s*b\s*a\s*j\s*o)\s*(?:T\s*e\s*r\s*m\s*i\s*n\s*a\s*l)\s*[No]*\s*\.?', documento, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)[1]
    documento = re.split(r'(?:alumn[oa]?s?\s*:?\s*|integrantes\s*:?\s*)', documento, flags = re.IGNORECASE)[0]
    documento = re.sub(r'\s|No\W*', '', documento)
    documento = documento.replace('_', '')
    documento = re.sub(r"[^a-zA-Z0-9]","",documento)    
    if re.search(r'^\d{4}[a-zA-Z]\d{3}$', documento):
        documento = documento[:4] + '-' + documento[4:]
    if not re.match(r'^\d{4}-[a-zA-Z]\d{3}$', documento):
        documento = re.split(r'\\', ruta)[-1]
        documento = re.split(r'\.', documento)[0]
        documento = re.sub(r"[^a-zA-Z0-9]","",documento)
        if re.search(r'^\d{4}[a-zA-Z]\d{3}$', documento):
            documento = documento[:4] + '-' + documento[4:]
        if not re.match(r'^\d{4}-[a-zA-Z]\d{3}$', documento):
            documento = '-'
    return documento.upper()

def get_abstract(documento):
    documento = re.split(r"Resumen", documento, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)[1]
    documento = re.split(r"Palabras?\s*clave", documento, maxsplit=1, flags = re.IGNORECASE | re.MULTILINE)[0]
    documento = re.sub(r'\n|^\s*[^a-zA-ZÁÉÍÓÚÜÑáéíóúüñ]+|\s{2,}|(^\s+|\s+$)', ' ', documento).strip()
    # eliminar todo lo que haya antes de la primera letra
    documento = re.sub(r'^[^a-zA-Z]+', '', documento)
    # if len(documento.split(' ')) > 400:
        # documento = '-'
    return documento

def get_keywords(documento):
    documento = re.split(r"Palabra[s]?\s*Clave[s]?\s*[-\._:]*\s*", documento, maxsplit=1, flags=re.IGNORECASE)[1]
    documento = re.split(r"\.", documento, maxsplit=1, flags=re.IGNORECASE)[0]
    documento = re.sub(r'^[^a-zA-ZÁÉÍÓÚÜÑáéíóúüñ]+|\n|\s{2,}', ' ', documento).strip()
    keywords = [keyword.strip() for keyword in re.split(r",", documento) if len(keyword.strip()) < 30]
    keywords = [re.sub(r'^[a-z]', lambda x: x.group(0).upper(), keyword) for keyword in keywords]
    return keywords
    
def get_objetivo(documento):
    i = 1
    objetivoGeneral = ''
    while len(objetivoGeneral) < 55:
        objetivo = re.split(r"[1-9]\.?\s*[-]?\s*(?:O\s*b\s*j\s*e\s*t\s*i\s*v\s*o)", documento, maxsplit=i, flags=re.IGNORECASE)[i]
        objetivo = re.split(r"[2-9]\.?\s*[-]?\s*(?:J\s*u\s*s\s*t\s*i\s*f\s*i\s*c\s*a\s*c\s*i\s*[oó]\s*n)", objetivo, maxsplit=1, flags=re.IGNORECASE)[0]
        objetivo = re.sub(r'\n|\s{2,}', ' ', objetivo).strip()

        if re.search(r"(?:objetivos?\s*)?(?:especí?i?ficos|particulare?s?|secundarios?)", objetivo, flags=re.IGNORECASE):
            objetivoGeneral, objetivosEspecificos = re.split(r"(?:objetivos?\s*)?(?:especí?i?ficos|particulare?s?|secundarios?)", objetivo, maxsplit=1, flags=re.IGNORECASE)
        else:
            objetivosEspecificos = '-'
            objetivoGeneral = objetivo

        if len(objetivoGeneral) < 50:
            i += 1

    objetivoGeneral = re.sub(r'^\s*(?:objetivos? general|objetivo|general)\s*', '', objetivoGeneral, flags=re.IGNORECASE)
    objetivoGeneral = re.sub(r'^[^a-zA-ZÁÉÍÓÚÜÑáéíóúüñ]+', '', objetivoGeneral)
    objetivosEspecificos = re.sub(r'^[^a-zA-ZÁÉÍÓÚÜÑáéíóúüñ]+', '', objetivosEspecificos)
    
    return objetivoGeneral, objetivosEspecificos

def get_areas(documento):
    directores = get_director(documento)
    areas = [''] * len(directores)

    for j, director in enumerate(directores):
        try:
            areas[j] = re.split(director, documento, flags=re.IGNORECASE | re.MULTILINE)[2]
        except IndexError:
            try:
                director_sin_acentos = re.sub(r'[áéíóú]', lambda m: m.group(0).lower(), director)
                areas[j] = re.split(director_sin_acentos, documento, flags=re.IGNORECASE | re.MULTILINE)[1]
            except IndexError:
                areas[j] = '-'

        if areas[j] != '-':
            areas[j] = re.split(r"Firma", areas[j], flags=re.IGNORECASE | re.MULTILINE)[0]
            if re.search(r'(?:Á|A)reas?\s*(?:de)?\s*inter(?:é|e)?s', areas[j], flags=re.IGNORECASE):
                areas[j] = re.split(r'(?:Á|A)reas?\s*(?:de)?\s*inter(?:é|e)?s', areas[j], flags=re.IGNORECASE | re.MULTILINE)[1]
                areas[j] = re.split(r"\.", areas[j], maxsplit=1, flags=re.IGNORECASE | re.MULTILINE)[0]
                areas[j] = re.sub(r'^[^a-zA-ZÁÉÍÓÚÜÑáéíóúüñ]+|\s{2,}|\n', ' ', areas[j]).strip()
                areas[j] = [i.strip() for i in re.split(r"\,", areas[j], flags=re.IGNORECASE | re.MULTILINE)]
            else:
                areas[j] = '-'
    areas = [[re.sub(r'^[a-zÁÉÍÓÚÜÑáéíóúüñ]', lambda x: x.group(0).upper(), i) for i in area] for area in areas]
    return areas

In [13]:
def get_all_info(documento, i):
    documento = eliminar_caracteres_especiales(documento)
    titulo = get_title(documento)
    autores = get_autores(documento)
    ntt= get_ttnum(documento, pdf_files[i])
    abstract = get_abstract(documento)
    keywords = get_keywords(documento)
    directores = get_director(documento)
    areas = get_areas(documento)
    objetivosG, objetivosE = get_objetivo(documento)
    rutas = pdf_files[i]

    return titulo, autores, ntt, directores, areas, keywords, abstract, objetivosG, objetivosE, rutas

In [14]:
protocolos = []
i=0
for i in range(len(pdf_files)):
    print(i + 1, '/', len(pdf_files), end='\r')
    protocolos.append(get_all_info(text[i], i))

In [17]:
# imprimir contenido de protocolos
# for i in range(10):
#     print(f"Protocolo {i+1}")
#     print(f"Titulo: {protocolos[i][0]}")
#     print(f"Autores: {protocolos[i][1]}")
#     print(f"Numero de TT: {protocolos[i][2]}")
#     print(f"Directores: {protocolos[i][3]}")
#     print(f"Areas: {protocolos[i][4]}")
#     print(f"Keywords: {protocolos[i][5]}")
#     print(f"Abstract: {protocolos[i][6]}")
#     print(f"Objetivo General: {protocolos[i][7]}")
#     print(f"Objetivos Especificos: {protocolos[i][8]}")
#     print(f"Rutas:  {protocolos[i][9]}")
#     print("--------------------------------------------------")

In [18]:
import pandas as pd
df = pd.DataFrame(protocolos, columns=['Título', 'Autores', 'Número de TT', 'Directores', 'Áreas', 'Palabras clave', 'Resumen', 'Objetivo general', 'Objetivos específicos', 'Ruta'])


In [19]:
# guardar df['Autores'] en un archivo xlsx
df.to_excel('prueba2.xlsx', index=False)

In [104]:
# df.to_excel('protocolos.xlsx', index=False)

In [3]:
data = pd.read_excel("protocolos.xlsx")

In [4]:
# dropear de data la columna de unnamed: 0 y ruta
data = data.drop(['Unnamed: 0', 'Ruta'], axis=1)

In [5]:
import spacy
nlp = spacy.load('es_core_news_sm')
tags = ['NOUN','ADJ', 'VERB']

def lematizar(textos: list):
    textos_lem = []
    for texto in textos:
        doc = nlp(str(texto))
        textos_lem.append(' '.join([token.lemma_ for token in doc if token.pos_ in tags]))
    return textos_lem

# lematizar titulos
titulos_lem = lematizar(data['Título'].tolist())

# lematizar resumenes
resumenes_lem = lematizar(data['Resumen'].tolist())

# lematizar objetivos generales
objetivosG_lem = lematizar(data['Objetivo general'].tolist())

# lematizar objetivos especificos
objetivosE_lem = lematizar(data['Objetivos específicos'].tolist())

# lematizar palabras clave
palabras_lem = lematizar(data['Palabras clave'].tolist())


In [35]:
resumenes_lem

['pretender',
 'desarrollar',
 'laboratorio',
 'virtual',
 'electrónica',
 'analógico',
 'utilizar',
 'conocimiento',
 'electrónico',
 'programación',
 'adquirido',
 'transcurso',
 'carrera',
 'propósito',
 'satisfacer',
 'demanda',
 'equipo',
 'ción',
 'encontrar',
 'laboratorio',
 'electrónica',
 'apoyandir',
 'tiempo',
 'ación',
 'espacio',
 'almacenamiento',
 'ocupar',
 'equipo',
 'tal',
 'manera',
 'portátil',
 'alumno',
 'utilizar él',
 'mayor',
 'comodidad',
 'eficiencia',
 'poder',
 'contribuir',
 'preservación',
 'ambiente',
 'resultar',
 'indispensable',
 'día',
 'debido',
 'vez',
 'menor',
 'área',
 'verd',
 'área',
 'cultivo',
 'volver',
 'productivo',
 'trabajo',
 'terminal',
 'proponer',
 'desarrollar',
 'invernadero',
 'colaborar',
 'proteger',
 'proceso',
 'crecimiento',
 'cultivo',
 'especie',
 'hacer él',
 'manera',
 'proponer',
 'desarrollar',
 'sistema',
 'controlar',
 'manera',
 'difús',
 'variable',
 'temperatura',
 'humedad',
 'ventilación',
 'censar',
 'ayuda',


DIRECTORES

In [173]:
# ABRIR XLSX CON NOMBRE Info TT 2023-B001.xlsx EN UN DATAFRAME
df1 = pd.read_excel('Info TT 2023-B001.xlsx', sheet_name='21-2')
df2 = pd.read_excel('Info TT 2023-B001.xlsx', sheet_name='22-1')
df3 = pd.read_excel('Info TT 2023-B001.xlsx', sheet_name='22-2')
df4 = pd.read_excel('Info TT 2023-B001.xlsx', sheet_name='23-1')

# CONCATENAR LOS DATAFRAMES
df = pd.concat([df1, df2, df3, df4], ignore_index=True)
df.head()

,PROF,DEPTO,UNIDAD APRENDIZAJE,ACADEMIA
0,Aguilar García Rafael,CIC,Paradigmas de Programación,CC
1,Aguilar García Rafael,CIC,Paradigmas de Programación,CC
2,Aguilar Sánchez Fernando,ISC,Fundamentos de Diseño Digital,SDIG
3,Aguilar Sánchez Fernando,ISC,Introducción a los Microcontroladores,SDIG
4,Aguilar Sánchez Fernando,ISC,Fundamentos de Diseño Digital,SDIG
